In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Nn.StrategyCriterion'




In [2]:
--- FUNCTIONS
--- Comparing with 2000
function compare_cfrs(tree_cfr1,tree_cfr2000)
    local difference = {}
    local table1 = tree_cfr1.total_strategies
    local table2 = tree_cfr2000.total_strategies
    
    for i = 1, #table1 do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table1[i] - table2[#table2]),2)))
    end
    return torch.FloatTensor(difference)
end
--- Comparing 2000 with itself
function get_convergence(tree_cfr)
    local difference = {}
    local table_cfr = tree_cfr.total_strategies
    local end_strategy = table_cfr[#table_cfr]
    
    for i = 1, #table_cfr do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table_cfr[i] - end_strategy),2)))
    end
    return torch.FloatTensor(difference)
end

In [3]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))

### 1. Compariaisons between CFR Solvers

In [4]:
--- Building CFR Solver
tree2000 = builder:build_tree(params)
tree2000_cfr = TreeCFR2()
tree2000_cfr:run_cfr(tree2000, starting_ranges,100)

In [5]:
tree2000_cfr:run_cfr(tree2000, starting_ranges,100)

In [6]:
--- Building CFR Solver
tree2001 = builder:build_tree(params)
tree2001_cfr = TreeCFR()
tree2001_cfr:run_cfr(tree2001, starting_ranges,100)

In [7]:
local tree_values = TreeValues()
tree_values:compute_values(tree2001)
print('Exploitability: ' .. tree2001.exploitability .. '[chips]' )

Exploitability: 11.352476596832[chips]


In [8]:
tree2001_cfr:run_cfr(tree2001, starting_ranges,100)

### 2. Comparision between different Starts

In [9]:
number_of_iterations = 800

In [10]:
--- Get Data from 2000 iterations
tree800 = builder:build_tree(params)
tree_data800 = TreeData(tree800)
tree800_cfr = TreeCFR()
print("Solver")
tree800_cfr:run_cfr(tree800, starting_ranges,number_of_iterations)
tree_data800:get_training_set(tree800,1)

Solver


### Training the Model

In [11]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model
-- Building trainer:
nn_trainer = NNTrainer(tree_data800,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [12]:
---current_criterion = nn.SmoothL1Criterion()
current_criterion = StrategyLoss()
train_loss,test_loss = nn_trainer:train(nn_model,current_criterion,opt,10000)

In [13]:
pred_strat = nn_model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})
true_strat =nn_trainer.all_data.targets
print(torch.mean(torch.abs(pred_strat- true_strat)))

0.0032879742351541


### No Warm Start

In [14]:
--- Building CFR Solver
treeNoWarm = builder:build_tree(params)
treeNoWarm_cfr = TreeCFR2()
print("Solver")
treeNoWarm_cfr:run_cfr(treeNoWarm, starting_ranges,number_of_iterations)

Solver


### Neural Net Start 

In [15]:
--- NNWarm Starting
treeNNWarm = builder:build_tree(params)
dataNN = TreeData(treeNNWarm)
dataNN:get_training_set(treeNNWarm,1)
dataNN:generate_new_initial_strategies(treeNNWarm,nn_trainer)

In [16]:
--- Building CFR Solver
treeNNWarm_cfr = TreeCFR2()
treeNNWarm_cfr:run_cfr(treeNNWarm, starting_ranges,number_of_iterations)

### Target Warm Start 

In [17]:
--- Target Warm Starting
treeTargWarm = builder:build_tree(params)
dataTarg = TreeData(treeTargWarm)
dataTarg:get_training_set(treeTargWarm,1)
dataTarg:warm_start_targets_and_regrets(treeTargWarm,tree800)

In [18]:
--- Building CFR Solver
treeTargWarm_cfr = TreeCFR2()
treeTargWarm_cfr:run_cfr(treeTargWarm, starting_ranges,number_of_iterations)

### 2000 Iterations

In [19]:
--- Building CFR Solver
tree2000 = builder:build_tree(params)
tree2000_cfr = TreeCFR()
tree2000_cfr:run_cfr(tree2000, starting_ranges,200)

### Graphical Comparaison

In [20]:
diff_NoW = compare_cfrs(treeNoWarm_cfr,tree2000_cfr)---:narrow(1,101,700)
diff_NNW = compare_cfrs(treeNNWarm_cfr,tree2000_cfr)---:narrow(1,101,700)
diff_TargW = compare_cfrs(treeTargWarm_cfr,tree2000_cfr)---:narrow(1,101,700)

In [41]:
local num_iter = torch.range(1,diff_NoW:size(1))
local plot = Plot()

plot:line(num_iter,diff_NoW,'red' ,'NW: Strategy without WS')
plot:line(num_iter,diff_NNW,'blue' ,'NN : Strategy initialized with NN')
plot:line(num_iter,diff_TargW,'green' ,'SNN : Strategy initialized with target values')
plot:legend(true):title('MSE between strategies and 200 iter CFR ')
plot:xaxis('number of iterations'):yaxis('Distance to end policy'):redraw()


### 3. Comparaision with 2000 CFR2

In [22]:
--- Get Data from 2000 iterations
tree802 = builder:build_tree(params)
tree_data802 = TreeData(tree802)
tree802_cfr = TreeCFR2()
tree802_cfr:run_cfr(tree802, starting_ranges,number_of_iterations)
tree_data802:get_training_set(tree802,1)

In [23]:
-- Building the neural net model
strategy_nn2 = StrategyNN()
nn_model2 = strategy_nn2.model
-- Building trainer:
nn_trainer2 = NNTrainer(tree_data802,nn_model2)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [24]:
current_criterion2 = nn.SmoothL1Criterion()
train_loss,test_loss = nn_trainer:train(nn_model2,current_criterion2,opt,10000)

In [25]:
--- No Warm Start
treeNoWarm2 = builder:build_tree(params)
treeNoWarm_cfr2 = TreeCFR2()
treeNoWarm_cfr2:run_cfr(treeNoWarm2, starting_ranges,number_of_iterations)

In [26]:
--- NNWarm Starting
treeNNWarm2 = builder:build_tree(params)
dataNN2 = TreeData(treeNNWarm2)
dataNN2:get_training_set(treeNNWarm2,1)
dataNN2:generate_new_initial_strategies(treeNNWarm2,nn_trainer2)

In [27]:
--- NNWarm CFR Solver
treeNNWarm_cfr2 = TreeCFR2()
treeNNWarm_cfr2:run_cfr(treeNNWarm2, starting_ranges,number_of_iterations)

In [42]:
--- Target Warm Starting
treeTargWarm2 = builder:build_tree(params)
dataTarg2 = TreeData(treeTargWarm2)
dataTarg2:get_training_set(treeTargWarm2,1)
dataTarg2:warm_start_targets_and_regrets(treeTargWarm2,tree802)

In [43]:
--- Building CFR Solver
treeTargWarm_cfr2 = TreeCFR2()
treeTargWarm_cfr2:run_cfr(treeTargWarm2, starting_ranges,number_of_iterations)

In [53]:
--- Building CFR Solver
tree2001 = builder:build_tree(params)
tree2001_cfr = TreeCFR2()
print("Solver")
tree2001_cfr:run_cfr(tree2001, starting_ranges,200)

Solver


In [54]:
diff_NoW2 = compare_cfrs(treeNoWarm_cfr2,tree2001_cfr)---:narrow(1,101,700)
diff_NNW2 = compare_cfrs(treeNNWarm_cfr2,tree2001_cfr)---:narrow(1,101,700)
diff_TargW2 = compare_cfrs(treeTargWarm_cfr2,tree2001_cfr)---:narrow(1,101,700)

In [55]:
local num_iter = torch.range(1,diff_NoW:size(1))
local plot = Plot()

plot:line(num_iter,diff_NoW2,'red' ,'NW: Strategy without WS')
plot:line(num_iter,diff_NNW2,'blue' ,'NN : Strategy initialized with NN')
plot:line(num_iter,diff_TargW2,'green' ,'SNN : Strategy initialized with target values')
plot:legend(true):title('MSE between strategies and 200 iter CFR ')
plot:xaxis('number of iterations'):yaxis('Distance to end policy'):redraw()


### Comparing two different strats

In [56]:
cfr_01 = compare_cfrs(tree2001_cfr,tree2000_cfr)---:narrow(1,101,700)
cfr_00 = compare_cfrs(tree2000_cfr,tree2000_cfr)---:narrow(1,101,700)

In [57]:
local num_iter = torch.range(1,#tree2001_cfr.total_strategies)
local plot = Plot()
plot:line(num_iter,cfr_01,'blue' ,'Solver modified')
plot:line(num_iter,cfr_00,'green' ,'Original Solver')
plot:legend(true):title('2000 iterations run vs Original Solver end Policy')
plot:xaxis('number of iterations'):yaxis('MSE Distance to end policy'):redraw()

### Second 2000 iterations

In [52]:
--- Comparing with 2000
function compare2000(tree_cfr1,tree_cfr2000)
    local difference = {}
    local table1 = tree_cfr1.total_strategies
    local table2 = tree_cfr2000.total_strategies
    
    for i = 1, #table1 do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table1[i] - table2[#table1]),2)))
    end
    return torch.FloatTensor(difference)
end

In [39]:
---diff2000 = get_convergence(tree2000_cfr):narrow(1,101,1899)
---diff2001 = get_convergence(tree2001_cfr):narrow(1,101,1899)

In [40]:
--[[local num_iter = torch.range(1,diff2000:size(1))
local plot = Plot()

plot:line(num_iter,diff2000,'red' ,'NW: Strategy without WS')
plot:line(num_iter,diff2001,'blue' ,'NW: Strategy without WS')
plot:legend(true):title('MSE between strategies and 2000 iter CFR ')
plot:draw()--]]